In [22]:
# Basic decorator to time function

def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        print(f'Run time: {elapsed : .6f}s')
        return result
    return inner

def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-2) + calc_fib_recurse(n-1)

def fib(n):
    return calc_fib_recurse(n)

In [10]:
fib = timed(fib)
fib(30)

Run time:  0.096804s
Run time:  0.097154s
Run time:  0.097175s
Run time:  0.097188s
Run time:  0.097201s
Run time:  0.097216s
Run time:  0.097229s
Run time:  0.097242s


832040

In [26]:
# we would have to add decorators that take parameters to be able to make an average of the 
# run time

def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (end - start)
        avg_run_time = total_elapsed / 10
        print(f'Avg run time: {total_elapsed : .6f}s')
        return result
    return inner

def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib)

In [27]:
fib(28)

Avg run time:  0.384610s


317811

In [30]:
# Now the problem is that in the previous version we had hardcoded our
# value, for us to use it we would have to pass it

def timed(fn, reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (end - start)
        avg_run_time = total_elapsed / reps
        print(f'Avg run time: {total_elapsed : .6f}s (ran {reps} times)')
        return result
    return inner

def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib, 10) # this works for this method of decorating not the @ approach
# That means that by using this approach we have lost the ability to use the
# @decorator since doing @decorator() means we are passing a function call.

In [31]:
fib(28)

Avg run time:  0.409068s (ran 10 times)


317811

In [33]:
# to be able to use the @ we would have to create a decorator factory
def dec(fn):
    print("running dec")

    def inner(*args, **kwargs):
        print("running inner")
        return fn(*args, **kwargs)
    return inner

@dec
def add(a, b):
    return a + b

# it runs dec, remember it is actually doing add = dec(add)


running dec


In [35]:
add(10, 2)

running inner


12

In [39]:
def dec_factory():
    print("Running dec_factory")

    def dec(fn):
        print("running dec")

        def inner(*args, **kwargs):
            print("running inner")
            return fn(*args, **kwargs)
        return inner
    return dec

# @dec_factory()
def fun():
    print("running fun")

fun = dec_factory()(fun)

fun() # dec_factory returns the decorator, the decorator returns the closure.

Running dec_factory
running dec
running inner
running fun


In [41]:
def dec_factory(a, b):
    print("Running dec_factory")

    def dec(fn):
        print("running dec")

        def inner(*args, **kwargs):
            print("running inner")
            print(f"{a}, {b}")
            return fn(*args, **kwargs)
        return inner
    return dec

dec = dec_factory(1,2)

@dec
def myfun():
    print("running my_func")

myfun()

Running dec_factory
running dec
running inner
1, 2
running my_func


In [43]:
@dec_factory(100, 200)
def myfun2():
    return("running my_func from the factory")

myfun2()

Running dec_factory
running dec
running inner
100, 200


'running my_func from the factory'

In [47]:
# now to apply it again to timed.
def timed(n):
    def dec(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(n):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (end - start)
            avg_run_time = total_elapsed / n
            print(f'Avg run time: {total_elapsed : .6f}s (ran {n} times)')
            return result
        return inner
    return dec

@timed(15)
def fib(n):
    return calc_fib_recurse(n)

fib(30)

Avg run time:  1.628283s (ran 15 times)


832040